In [1]:
import pandas as pd
import numpy as np
import missingno as msno

import matplotlib
import matplotlib.pyplot as plt

import seaborn as sns

import math

import sklearn
from sklearn.metrics import roc_auc_score
from sklearn.dummy import DummyClassifier
from sklearn.model_selection import KFold, StratifiedKFold
from sklearn.linear_model import LogisticRegression
from sklearn.model_selection import StratifiedKFold
from sklearn.preprocessing import StandardScaler

import time

In [2]:
from sklearn.compose import ColumnTransformer
from sklearn.preprocessing import OneHotEncoder

In [68]:
# On supprime les avertissements nous indiquant que l'on change les valeurs de notre jeu de données d'origine
pd.options.mode.chained_assignment = None

In [3]:
# On charge nos deux jeux de données
df_test = pd.read_csv('application_test.csv')
df = pd.read_csv('application_train.csv')
bureau = pd.read_csv('bureau.csv')
bureau_balance = pd.read_csv('bureau_balance.csv')
ccb = pd.read_csv('credit_card_balance.csv')
install_pay = pd.read_csv('installments_payments.csv')
pos_cash = pd.read_csv('POS_CASH_balance.csv')
prev_app = pd.read_csv('previous_application.csv')

# Fonctions

In [4]:
# # One-hot encoding for categorical columns with get_dummies
# categorical_columns_total = []

# def one_hot_encoder(df, nan_as_category = True):
#     original_columns = list(df.columns)
#     categorical_columns = [col for col in df.columns if df[col].dtype == 'object']
#     categorical_columns_total.append(categorical_columns)
#     df = pd.get_dummies(df, columns= categorical_columns, dummy_na= nan_as_category)
#     new_columns = [c for c in df.columns if c not in original_columns]
#     return df, new_columns

In [5]:
# One-hot encoding for categorical columns with get_dummies
not_categorical_columns_total = []

def preprocessing(df, cols_to_change):

    original_columns = list(df.columns)
    categorical_columns = [col for col in df.columns if df[col].dtype == 'object']

    for col in cols_to_change:
        df[col] = df[col].astype('object')
        
    not_categorical_columns = [col for col in df.columns if df[col].dtype != 'object']
    not_categorical_columns_total.append(not_categorical_columns)

    scaler = StandardScaler()
    df[not_categorical_columns] = scaler.fit_transform(df[not_categorical_columns])

    df = pd.get_dummies(df, columns=categorical_columns)
    new_columns = [c for c in df.columns if c not in original_columns]
    return df, new_columns, not_categorical_columns

## Application (1/6)

In [6]:
df.shape

(307511, 122)

In [7]:
df_cols_drop = ['REGION_POPULATION_RELATIVE', 'DAYS_REGISTRATION','DAYS_ID_PUBLISH',
                'FLAG_MOBIL', 'FLAG_EMP_PHONE', 'FLAG_WORK_PHONE', 'FLAG_PHONE', 'FLAG_EMAIL',
                'REG_REGION_NOT_LIVE_REGION', 'REG_REGION_NOT_WORK_REGION', 'LIVE_REGION_NOT_WORK_REGION',
                'REG_CITY_NOT_LIVE_CITY', 'REG_CITY_NOT_WORK_CITY', 'LIVE_CITY_NOT_WORK_CITY',
                'APARTMENTS_AVG', 'BASEMENTAREA_AVG', 'YEARS_BEGINEXPLUATATION_AVG', 'YEARS_BUILD_AVG',
                'COMMONAREA_AVG', 'ELEVATORS_AVG', 'ENTRANCES_AVG', 'FLOORSMAX_AVG', 'FLOORSMIN_AVG',
                'LANDAREA_AVG', 'LIVINGAPARTMENTS_AVG', 'LIVINGAREA_AVG', 'NONLIVINGAPARTMENTS_AVG',
                'NONLIVINGAREA_AVG', 'APARTMENTS_MODE', 'BASEMENTAREA_MODE', 'YEARS_BEGINEXPLUATATION_MODE',
                'YEARS_BUILD_MODE', 'COMMONAREA_MODE', 'ELEVATORS_MODE', 'ENTRANCES_MODE', 'FLOORSMAX_MODE',
                'FLOORSMIN_MODE', 'LANDAREA_MODE', 'LIVINGAPARTMENTS_MODE', 'LIVINGAREA_MODE',
                'NONLIVINGAPARTMENTS_MODE', 'NONLIVINGAREA_MODE', 'APARTMENTS_MEDI', 'BASEMENTAREA_MEDI',
                'YEARS_BEGINEXPLUATATION_MEDI', 'YEARS_BUILD_MEDI', 'COMMONAREA_MEDI', 'ELEVATORS_MEDI',
                'ENTRANCES_MEDI', 'FLOORSMAX_MEDI', 'FLOORSMIN_MEDI', 'LANDAREA_MEDI', 'LIVINGAPARTMENTS_MEDI',
                'LIVINGAREA_MEDI', 'NONLIVINGAPARTMENTS_MEDI', 'NONLIVINGAREA_MEDI', 'FONDKAPREMONT_MODE',
                'HOUSETYPE_MODE', 'TOTALAREA_MODE', 'WALLSMATERIAL_MODE', 'EMERGENCYSTATE_MODE']

df_cols_voir = ['NAME_TYPE_SUITE', 'CNT_CHILDREN', 'AMT_GOODS_PRICE','NAME_FAMILY_STATUS', 'OWN_CAR_AGE',
               'REGION_RATING_CLIENT', 'REGION_RATING_CLIENT_W_CITY', 'WEEKDAY_APPR_PROCESS_START',
               'HOUR_APPR_PROCESS_START', 'OBS_30_CNT_SOCIAL_CIRCLE', 'DEF_30_CNT_SOCIAL_CIRCLE',
                'OBS_60_CNT_SOCIAL_CIRCLE', 'DEF_60_CNT_SOCIAL_CIRCLE', 'DAYS_LAST_PHONE_CHANGE',
                'FLAG_DOCUMENT_2', 'FLAG_DOCUMENT_3', 'FLAG_DOCUMENT_4', 'FLAG_DOCUMENT_5',
                'FLAG_DOCUMENT_6', 'FLAG_DOCUMENT_7', 'FLAG_DOCUMENT_8',
               'FLAG_DOCUMENT_9', 'FLAG_DOCUMENT_10', 'FLAG_DOCUMENT_11', 'FLAG_DOCUMENT_12', 'FLAG_DOCUMENT_13',
               'FLAG_DOCUMENT_14', 'FLAG_DOCUMENT_15', 'FLAG_DOCUMENT_16', 'FLAG_DOCUMENT_17', 'FLAG_DOCUMENT_18',
               'FLAG_DOCUMENT_19', 'FLAG_DOCUMENT_20', 'FLAG_DOCUMENT_21', 'AMT_REQ_CREDIT_BUREAU_HOUR',
               'AMT_REQ_CREDIT_BUREAU_DAY', 'AMT_REQ_CREDIT_BUREAU_WEEK', 'AMT_REQ_CREDIT_BUREAU_MON',
               'AMT_REQ_CREDIT_BUREAU_QRT', 'AMT_REQ_CREDIT_BUREAU_YEAR']

# On supprime les colonnes qui ne nous intéressent pas
df = df.drop(columns = df_cols_drop)
df_test = df_test.drop(columns = df_cols_drop)

# Pour l'instant on supprime les colonnes où on ne sait pas trop
df = df.drop(columns = df_cols_voir)
df_test = df_test.drop(columns = df_cols_voir)

# Taille des jeux de données après les changements
df.shape

(307511, 21)

In [8]:
df.columns

Index(['SK_ID_CURR', 'TARGET', 'NAME_CONTRACT_TYPE', 'CODE_GENDER',
       'FLAG_OWN_CAR', 'FLAG_OWN_REALTY', 'AMT_INCOME_TOTAL', 'AMT_CREDIT',
       'AMT_ANNUITY', 'NAME_INCOME_TYPE', 'NAME_EDUCATION_TYPE',
       'NAME_HOUSING_TYPE', 'DAYS_BIRTH', 'DAYS_EMPLOYED', 'FLAG_CONT_MOBILE',
       'OCCUPATION_TYPE', 'CNT_FAM_MEMBERS', 'ORGANIZATION_TYPE',
       'EXT_SOURCE_1', 'EXT_SOURCE_2', 'EXT_SOURCE_3'],
      dtype='object')

In [9]:
# Read data and merge
print("Taille Train: {}, taille test: {}".format(len(df), len(df_test)))
df = df.append(df_test).reset_index()

# On append train et test pour faire les mêmes preprocessing
# On les redivise après

# Optional: Remove 4 applications with XNA CODE_GENDER (train set)
df = df[df['CODE_GENDER'] != 'XNA']

# Categorical features with Binary encode (0 or 1; two categories)
for binary_feat in ['CODE_GENDER', 'FLAG_OWN_CAR', 'FLAG_OWN_REALTY']:
    df[binary_feat], uniques = pd.factorize(df[binary_feat])

cols_df_change = ['SK_ID_CURR', 'TARGET', 'CODE_GENDER', 'FLAG_OWN_CAR', 'FLAG_OWN_REALTY']
# Categorical features with One-Hot encode
df, cat_cols, not_categorical_columns_df = preprocessing(df, cols_to_change=cols_df_change)

# NaN values for DAYS_EMPLOYED: 365.243 -> nan
df['DAYS_EMPLOYED'].replace(365243, np.nan, inplace= True)

# Some simple new features (percentages)
df['DAYS_EMPLOYED_PERC'] = df['DAYS_EMPLOYED'] / df['DAYS_BIRTH']

# Taux d'endettement plutôt ?
df['INCOME_CREDIT_PERC'] = df['AMT_INCOME_TOTAL'] / df['AMT_CREDIT']
df['CREDIT_INCOME_PERC'] = df['AMT_CREDIT'] / df['AMT_INCOME_TOTAL'] # Taux d'endettement

df['INCOME_PER_PERSON'] = df['AMT_INCOME_TOTAL'] / df['CNT_FAM_MEMBERS']
df['ANNUITY_INCOME_PERC'] = df['AMT_ANNUITY'] / df['AMT_INCOME_TOTAL']
df['PAYMENT_RATE'] = df['AMT_ANNUITY'] / df['AMT_CREDIT']

Taille Train: 307511, taille test: 48744


In [10]:
df.head(2)

,index,SK_ID_CURR,TARGET,CODE_GENDER,FLAG_OWN_CAR,FLAG_OWN_REALTY,AMT_INCOME_TOTAL,AMT_CREDIT,AMT_ANNUITY,DAYS_BIRTH,...,ORGANIZATION_TYPE_Transport: type 3,ORGANIZATION_TYPE_Transport: type 4,ORGANIZATION_TYPE_University,ORGANIZATION_TYPE_XNA,DAYS_EMPLOYED_PERC,INCOME_CREDIT_PERC,CREDIT_INCOME_PERC,INCOME_PER_PERSON,ANNUITY_INCOME_PERC,PAYMENT_RATE
0,-1.449816,100002,1.0,0,0,0,0.144890,-0.454493,-0.184971,1.509656,...,0,0,0,0,-0.303631,-0.318796,-3.136803,-0.114208,-1.276625,0.406983
1,-1.449806,100003,0.0,1,0,1,0.446893,1.770419,0.561525,-0.166038,...,0,0,0,0,2.784112,0.252422,3.961614,-2.672016,1.256507,0.317171


In [11]:
df.isnull().sum()

index                      0
SK_ID_CURR                 0
TARGET                 48744
CODE_GENDER                0
FLAG_OWN_CAR               0
                       ...  
INCOME_CREDIT_PERC         0
CREDIT_INCOME_PERC         0
INCOME_PER_PERSON          2
ANNUITY_INCOME_PERC       36
PAYMENT_RATE              36
Length: 119, dtype: int64

In [12]:
df_nan = df.isnull().mean()
df_nan = df_nan.sort_values(ascending=False)
liste_nan = []

for x, y in zip(df_nan.index, df_nan):
    liste_nan.append((x, y))

liste_nan[:10]

[('EXT_SOURCE_1', 0.5443016300305122),
 ('EXT_SOURCE_3', 0.19546050397051518),
 ('TARGET', 0.1368248790880587),
 ('EXT_SOURCE_2', 0.0018750824559088956),
 ('PAYMENT_RATE', 0.0001010523479232339),
 ('ANNUITY_INCOME_PERC', 0.0001010523479232339),
 ('AMT_ANNUITY', 0.0001010523479232339),
 ('INCOME_PER_PERSON', 5.61401932906855e-06),
 ('CNT_FAM_MEMBERS', 5.61401932906855e-06),
 ('ORGANIZATION_TYPE_Industry: type 7', 0.0)]

In [13]:
# On remplace par la valeur médiane
cols = ['DAYS_EMPLOYED', 'DAYS_EMPLOYED_PERC', 'AMT_ANNUITY',
        'PAYMENT_RATE', 'ANNUITY_INCOME_PERC', 'INCOME_PER_PERSON',
        'CNT_FAM_MEMBERS']

for col in cols:
    df[col] = df[col].fillna(df[col].median())

In [14]:
df.isnull().mean()

index                  0.000000
SK_ID_CURR             0.000000
TARGET                 0.136825
CODE_GENDER            0.000000
FLAG_OWN_CAR           0.000000
                         ...   
INCOME_CREDIT_PERC     0.000000
CREDIT_INCOME_PERC     0.000000
INCOME_PER_PERSON      0.000000
ANNUITY_INCOME_PERC    0.000000
PAYMENT_RATE           0.000000
Length: 119, dtype: float64

In [15]:
df.shape

(356251, 119)

In [16]:
for col in df.columns.drop('TARGET'):
    df[col].replace([np.inf, -np.inf], np.nan, inplace=True)
    df[col].dropna(inplace=True)

df.shape

(356251, 119)

In [17]:
df['SK_ID_CURR'].nunique()

356251

## Bureau & bureau_balance (2/6)

In [18]:
# Pas de col à supprimer a priori
# bb_cols_drop = []

bb_cols_voir = ['CREDIT_CURRENCY', 'DAYS_CREDIT_UPDATE', 'DAYS_ENDDATE_FACT']

# Pour l'instant on supprime les colonnes où on ne sait pas trop
# bureau = bureau.drop(columns = bb_cols_drop)
bureau = bureau.drop(columns = bb_cols_voir)

In [19]:
bureau.dtypes

SK_ID_CURR                  int64
SK_ID_BUREAU                int64
CREDIT_ACTIVE              object
DAYS_CREDIT                 int64
CREDIT_DAY_OVERDUE          int64
DAYS_CREDIT_ENDDATE       float64
AMT_CREDIT_MAX_OVERDUE    float64
CNT_CREDIT_PROLONG          int64
AMT_CREDIT_SUM            float64
AMT_CREDIT_SUM_DEBT       float64
AMT_CREDIT_SUM_LIMIT      float64
AMT_CREDIT_SUM_OVERDUE    float64
CREDIT_TYPE                object
AMT_ANNUITY               float64
dtype: object

In [20]:
bureau_balance.dtypes

SK_ID_BUREAU       int64
MONTHS_BALANCE     int64
STATUS            object
dtype: object

In [21]:
cols_bb_change = ['SK_ID_BUREAU']
bureau_balance, bb_cat, not_categorical_columns_bb = preprocessing(bureau_balance, cols_to_change=cols_bb_change)

cols_bureau_change = ['SK_ID_CURR', 'SK_ID_BUREAU']
bureau, bureau_cat, not_categorical_columns_bureau = preprocessing(bureau, cols_to_change=cols_bureau_change)
    
# Bureau balance: Perform aggregations and merge with bureau.csv
bb_aggregations = {'MONTHS_BALANCE': ['min', 'max', 'size']}

for col in bb_cat:
    bb_aggregations[col] = ['mean']
bb_agg = bureau_balance.groupby('SK_ID_BUREAU').agg(bb_aggregations)
bb_agg.columns = pd.Index([e[0] + "_" + e[1].upper() for e in bb_agg.columns.tolist()])
bureau = bureau.join(bb_agg, how='left', on='SK_ID_BUREAU')
bureau.drop(['SK_ID_BUREAU'], axis=1, inplace= True)
    
# Bureau and bureau_balance numeric features
num_aggregations = {
    'DAYS_CREDIT': ['min', 'max', 'mean', 'var'],
    'DAYS_CREDIT_ENDDATE': ['min', 'max', 'mean'],
    'CREDIT_DAY_OVERDUE': ['max', 'mean'],
    'AMT_CREDIT_MAX_OVERDUE': ['mean'],
    'AMT_CREDIT_SUM': ['max', 'mean', 'sum'],
    'AMT_CREDIT_SUM_DEBT': ['max', 'mean', 'sum'],
    'AMT_CREDIT_SUM_OVERDUE': ['mean'],
    'AMT_CREDIT_SUM_LIMIT': ['mean', 'sum'],
    'AMT_ANNUITY': ['max', 'mean'],
    'CNT_CREDIT_PROLONG': ['sum'],
    'MONTHS_BALANCE_MIN': ['min'],
    'MONTHS_BALANCE_MAX': ['max'],
    'MONTHS_BALANCE_SIZE': ['mean', 'sum']
}
# 'DAYS_CREDIT_UPDATE': ['mean'],

# Bureau and bureau_balance categorical features
cat_aggregations = {}
for cat in bureau_cat: cat_aggregations[cat] = ['mean']
for cat in bb_cat: cat_aggregations[cat + "_MEAN"] = ['mean']

bureau_agg = bureau.groupby('SK_ID_CURR').agg({**num_aggregations, **cat_aggregations})
bureau_agg.columns = pd.Index(['BURO_' + e[0] + "_" + e[1].upper() for e in bureau_agg.columns.tolist()])

# Bureau: Active credits - using only numerical aggregations
active = bureau[bureau['CREDIT_ACTIVE_Active'] == 1]
active_agg = active.groupby('SK_ID_CURR').agg(num_aggregations)
active_agg.columns = pd.Index(['ACTIVE_' + e[0] + "_" + e[1].upper() for e in active_agg.columns.tolist()])
bureau_agg = bureau_agg.join(active_agg, how='left', on='SK_ID_CURR')

# Bureau: Closed credits - using only numerical aggregations
closed = bureau[bureau['CREDIT_ACTIVE_Closed'] == 1]
closed_agg = closed.groupby('SK_ID_CURR').agg(num_aggregations)
closed_agg.columns = pd.Index(['CLOSED_' + e[0] + "_" + e[1].upper() for e in closed_agg.columns.tolist()])
bureau_agg = bureau_agg.join(closed_agg, how='left', on='SK_ID_CURR')

In [22]:
bureau_agg.head(2)

,BURO_DAYS_CREDIT_MIN,BURO_DAYS_CREDIT_MAX,BURO_DAYS_CREDIT_MEAN,BURO_DAYS_CREDIT_VAR,BURO_DAYS_CREDIT_ENDDATE_MIN,BURO_DAYS_CREDIT_ENDDATE_MAX,BURO_DAYS_CREDIT_ENDDATE_MEAN,BURO_CREDIT_DAY_OVERDUE_MAX,BURO_CREDIT_DAY_OVERDUE_MEAN,BURO_AMT_CREDIT_MAX_OVERDUE_MEAN,...,CLOSED_AMT_CREDIT_SUM_OVERDUE_MEAN,CLOSED_AMT_CREDIT_SUM_LIMIT_MEAN,CLOSED_AMT_CREDIT_SUM_LIMIT_SUM,CLOSED_AMT_ANNUITY_MAX,CLOSED_AMT_ANNUITY_MEAN,CLOSED_CNT_CREDIT_PROLONG_SUM,CLOSED_MONTHS_BALANCE_MIN_MIN,CLOSED_MONTHS_BALANCE_MAX_MAX,CLOSED_MONTHS_BALANCE_SIZE_MEAN,CLOSED_MONTHS_BALANCE_SIZE_SUM
SK_ID_CURR,,,,,,,,,,,,,,,,,,,,,
100001,-0.540633,1.374693,0.511979,0.379644,-0.368329,0.253790,-0.085717,-0.022388,-0.022388,NaN,...,-0.006385,-0.138335,-0.553341,-0.048224,-0.048224,-0.266479,-0.848889,1.288176,35.0,140.0
100002,-0.370857,1.306783,0.337173,0.294407,-0.316870,0.053959,-0.172102,-0.022388,-0.022388,-0.010408,...,-0.006385,-0.138335,-0.276671,-0.048224,-0.048224,-0.399718,-0.681276,1.288176,15.0,90.0


In [23]:
bureau_agg.isnull().sum()

BURO_DAYS_CREDIT_MIN                    0
BURO_DAYS_CREDIT_MAX                    0
BURO_DAYS_CREDIT_MEAN                   0
BURO_DAYS_CREDIT_VAR                41520
BURO_DAYS_CREDIT_ENDDATE_MIN         2585
                                    ...  
CLOSED_CNT_CREDIT_PROLONG_SUM       37886
CLOSED_MONTHS_BALANCE_MIN_MIN      187316
CLOSED_MONTHS_BALANCE_MAX_MAX      187316
CLOSED_MONTHS_BALANCE_SIZE_MEAN    187316
CLOSED_MONTHS_BALANCE_SIZE_SUM      37886
Length: 105, dtype: int64

In [24]:
for col in bureau_agg.columns:
    if 'CLOSED' in col:
        bureau_agg[col] = bureau_agg[col].fillna(0)
    
    elif 'ACTIVE' in col:
        bureau_agg[col] = bureau_agg[col].fillna(0)
    
    elif 'BURO' in col:
        bureau_agg[col] = bureau_agg[col].fillna(bureau_agg[col].median())

In [25]:
bureau_agg.isnull().sum().max()

0

In [26]:
bureau_agg.shape

(305811, 105)

In [27]:
bureau_agg.replace([np.inf, -np.inf], np.nan, inplace=True)
bureau_agg.dropna(inplace=True)

bureau_agg.shape

(305811, 105)

## Previous application (3/6)

In [28]:
# Beaucoup de données où ne connait pas le vendeur / il n'y en a pas
prev_cols_drop = ['NAME_SELLER_INDUSTRY']

prev_cols_voir = ['AMT_GOODS_PRICE', 'CHANNEL_TYPE','HOUR_APPR_PROCESS_START',
                  'NAME_TYPE_SUITE','NFLAG_LAST_APPL_IN_DAY','SELLERPLACE_AREA',
                  'WEEKDAY_APPR_PROCESS_START']

# On supprime les colonnes choisies
prev_app = prev_app.drop(columns = prev_cols_drop)
prev_app = prev_app.drop(columns = prev_cols_voir)

In [29]:
# Si on garde "NAME_TYPE_SUITE"
# for row in prev_app.index:   
    
#     if prev_app.loc[row, 'NAME_TYPE_SUITE'] == 'Other_A':
#         prev_app.loc[row, 'NAME_TYPE_SUITE'] = 'Other'

#     if prev_app.loc[row, 'NAME_TYPE_SUITE'] == 'Other_B':
#         prev_app.loc[row, 'NAME_TYPE_SUITE'] = 'Other'

In [30]:
prev_app.dtypes

SK_ID_PREV                       int64
SK_ID_CURR                       int64
NAME_CONTRACT_TYPE              object
AMT_ANNUITY                    float64
AMT_APPLICATION                float64
AMT_CREDIT                     float64
AMT_DOWN_PAYMENT               float64
FLAG_LAST_APPL_PER_CONTRACT     object
RATE_DOWN_PAYMENT              float64
RATE_INTEREST_PRIMARY          float64
RATE_INTEREST_PRIVILEGED       float64
NAME_CASH_LOAN_PURPOSE          object
NAME_CONTRACT_STATUS            object
DAYS_DECISION                    int64
NAME_PAYMENT_TYPE               object
CODE_REJECT_REASON              object
NAME_CLIENT_TYPE                object
NAME_GOODS_CATEGORY             object
NAME_PORTFOLIO                  object
NAME_PRODUCT_TYPE               object
CNT_PAYMENT                    float64
NAME_YIELD_GROUP                object
PRODUCT_COMBINATION             object
DAYS_FIRST_DRAWING             float64
DAYS_FIRST_DUE                 float64
DAYS_LAST_DUE_1ST_VERSION

In [31]:
# Preprocess previous_applications.csv
cols_prev_change = ['SK_ID_PREV', 'SK_ID_CURR']
prev_app, cat_cols, not_categorical_columns_prev = preprocessing(prev_app, cols_to_change=cols_prev_change)

# Days 365.243 values -> nan
prev_app['DAYS_FIRST_DRAWING'].replace(365243, np.nan, inplace= True)
prev_app['DAYS_FIRST_DUE'].replace(365243, np.nan, inplace= True)
prev_app['DAYS_LAST_DUE_1ST_VERSION'].replace(365243, np.nan, inplace= True)
prev_app['DAYS_LAST_DUE'].replace(365243, np.nan, inplace= True)
prev_app['DAYS_TERMINATION'].replace(365243, np.nan, inplace= True)

# Add feature: value ask / value received percentage
prev_app['APP_CREDIT_PERC'] = prev_app['AMT_APPLICATION'] / prev_app['AMT_CREDIT']

# Previous applications numeric features
# Si on garde les "à voir"

# num_aggregations = {
#     'AMT_ANNUITY': ['min', 'max', 'mean'],
#     'AMT_APPLICATION': ['min', 'max', 'mean'],
#     'AMT_CREDIT': ['min', 'max', 'mean'],
#     'APP_CREDIT_PERC': ['min', 'max', 'mean', 'var'],
#     'AMT_DOWN_PAYMENT': ['min', 'max', 'mean'],
#     'AMT_GOODS_PRICE': ['min', 'max', 'mean'],
#     'HOUR_APPR_PROCESS_START': ['min', 'max', 'mean'],
#     'RATE_DOWN_PAYMENT': ['min', 'max', 'mean'],
#     'DAYS_DECISION': ['min', 'max', 'mean'],
#     'CNT_PAYMENT': ['mean', 'sum'],
# }


num_aggregations = {
    'AMT_ANNUITY': ['min', 'max', 'mean'],
    'AMT_APPLICATION': ['min', 'max', 'mean'],
    'AMT_CREDIT': ['min', 'max', 'mean'],
    'APP_CREDIT_PERC': ['min', 'max', 'mean', 'var'],
    'AMT_DOWN_PAYMENT': ['min', 'max', 'mean'],
    'RATE_DOWN_PAYMENT': ['min', 'max', 'mean'],
    'DAYS_DECISION': ['min', 'max', 'mean'],
    'CNT_PAYMENT': ['mean', 'sum'],
}

# Previous applications categorical features
cat_aggregations = {}
for cat in cat_cols:
    cat_aggregations[cat] = ['mean']
    
prev_agg = prev_app.groupby('SK_ID_CURR').agg({**num_aggregations, **cat_aggregations})
prev_agg.columns = pd.Index(['PREV_' + e[0] + "_" + e[1].upper() for e in prev_agg.columns.tolist()])

# Previous Applications: Approved Applications - only numerical features
approved = prev_app[prev_app['NAME_CONTRACT_STATUS_Approved'] == 1]
approved_agg = approved.groupby('SK_ID_CURR').agg(num_aggregations)
approved_agg.columns = pd.Index(['APPROVED_' + e[0] + "_" + e[1].upper() for e in approved_agg.columns.tolist()])
prev_agg = prev_agg.join(approved_agg, how='left', on='SK_ID_CURR')

# Previous Applications: Refused Applications - only numerical features
refused = prev_app[prev_app['NAME_CONTRACT_STATUS_Refused'] == 1]
refused_agg = refused.groupby('SK_ID_CURR').agg(num_aggregations)
refused_agg.columns = pd.Index(['REFUSED_' + e[0] + "_" + e[1].upper() for e in refused_agg.columns.tolist()])
prev_agg = prev_agg.join(refused_agg, how='left', on='SK_ID_CURR')

In [32]:
prev_agg.isnull().sum()

PREV_AMT_ANNUITY_MIN             480
PREV_AMT_ANNUITY_MAX             480
PREV_AMT_ANNUITY_MEAN            480
PREV_AMT_APPLICATION_MIN           0
PREV_AMT_APPLICATION_MAX           0
                               ...  
REFUSED_DAYS_DECISION_MIN     220580
REFUSED_DAYS_DECISION_MAX     220580
REFUSED_DAYS_DECISION_MEAN    220580
REFUSED_CNT_PAYMENT_MEAN      230761
REFUSED_CNT_PAYMENT_SUM       220580
Length: 182, dtype: int64

In [33]:
for col in prev_agg.columns:
    if 'REFUSED' in col:
        prev_agg[col] = prev_agg[col].fillna(0)
    
    elif 'APPROVED' in col:
        prev_agg[col] = prev_agg[col].fillna(0)
    
    elif 'PREV' in col:
        prev_agg[col] = prev_agg[col].fillna(prev_agg[col].median())

In [34]:
prev_agg.isnull().sum().max()

0

In [35]:
prev_agg.shape

(338857, 182)

In [36]:
prev_agg.replace([np.inf, -np.inf], np.nan, inplace=True)
prev_agg.dropna(inplace=True)

prev_agg.shape

(338857, 182)

## POS balance (4/6)

In [37]:
# A priori on garde tout

# pos_cols_drop = []

# pos_cols_voir = []

# Pour l'instant on supprime les colonnes où on ne sait pas trop
# pos_cash = pos_cash.drop(columns = pos_cols_drop)
# pos_cash = pos_cash.drop(columns = pos_cols_voir)

In [38]:
pos_cash.dtypes

SK_ID_PREV                 int64
SK_ID_CURR                 int64
MONTHS_BALANCE             int64
CNT_INSTALMENT           float64
CNT_INSTALMENT_FUTURE    float64
NAME_CONTRACT_STATUS      object
SK_DPD                     int64
SK_DPD_DEF                 int64
dtype: object

In [39]:
# Preprocess POS_CASH_balance.csv
cols_pos_change = ['SK_ID_PREV', 'SK_ID_CURR']
pos_cash, cat_cols, not_categorical_columns_pos = preprocessing(pos_cash, cols_to_change=cols_pos_change)

# Features
aggregations = {
    'MONTHS_BALANCE': ['max', 'mean', 'size'],
    'SK_DPD': ['max', 'mean'],
    'SK_DPD_DEF': ['max', 'mean']
}
for cat in cat_cols:
    aggregations[cat] = ['mean']
    
pos_agg = pos_cash.groupby('SK_ID_CURR').agg(aggregations)
pos_agg.columns = pd.Index(['POS_' + e[0] + "_" + e[1].upper() for e in pos_agg.columns.tolist()])

# Count pos cash accounts
pos_agg['POS_COUNT'] = pos_cash.groupby('SK_ID_CURR').size()

In [42]:
pos_agg.isnull().sum()

POS_MONTHS_BALANCE_MAX                                 0
POS_MONTHS_BALANCE_MEAN                                0
POS_MONTHS_BALANCE_SIZE                                0
POS_SK_DPD_MAX                                         0
POS_SK_DPD_MEAN                                        0
POS_SK_DPD_DEF_MAX                                     0
POS_SK_DPD_DEF_MEAN                                    0
POS_NAME_CONTRACT_STATUS_Active_MEAN                   0
POS_NAME_CONTRACT_STATUS_Amortized debt_MEAN           0
POS_NAME_CONTRACT_STATUS_Approved_MEAN                 0
POS_NAME_CONTRACT_STATUS_Canceled_MEAN                 0
POS_NAME_CONTRACT_STATUS_Completed_MEAN                0
POS_NAME_CONTRACT_STATUS_Demand_MEAN                   0
POS_NAME_CONTRACT_STATUS_Returned to the store_MEAN    0
POS_NAME_CONTRACT_STATUS_Signed_MEAN                   0
POS_NAME_CONTRACT_STATUS_XNA_MEAN                      0
POS_COUNT                                              0
dtype: int64

In [43]:
pos_agg.shape

(337252, 17)

In [44]:
pos_agg.replace([np.inf, -np.inf], np.nan, inplace=True)
pos_agg.dropna(inplace=True)

pos_agg.shape

(337252, 17)

## Installments payments (5/6)

In [45]:
# ins_cols_drop = []

# ins_cols_voir = []

# Pour l'instant on supprime les colonnes où on ne sait pas trop
# install_pay = install_pay.drop(columns = ins_cols_drop)
# install_pay = install_pay.drop(columns = ins_cols_voir)

In [46]:
install_pay.isnull().sum()

SK_ID_PREV                   0
SK_ID_CURR                   0
NUM_INSTALMENT_VERSION       0
NUM_INSTALMENT_NUMBER        0
DAYS_INSTALMENT              0
DAYS_ENTRY_PAYMENT        2905
AMT_INSTALMENT               0
AMT_PAYMENT               2905
dtype: int64

In [47]:
for col in install_pay.columns:
    install_pay[col] = install_pay[col].fillna(install_pay[col].median())

In [48]:
install_pay.dtypes

SK_ID_PREV                  int64
SK_ID_CURR                  int64
NUM_INSTALMENT_VERSION    float64
NUM_INSTALMENT_NUMBER       int64
DAYS_INSTALMENT           float64
DAYS_ENTRY_PAYMENT        float64
AMT_INSTALMENT            float64
AMT_PAYMENT               float64
dtype: object

In [49]:
# Preprocess installments_payments.csv
cols_ins_change = ['SK_ID_PREV', 'SK_ID_CURR']
install_pay, cat_cols, not_categorical_columns_ins = preprocessing(install_pay, cols_to_change=cols_ins_change)

# Percentage and difference paid in each installment (amount paid and installment value)
install_pay['PAYMENT_PERC'] = install_pay['AMT_PAYMENT'] / install_pay['AMT_INSTALMENT']
install_pay['PAYMENT_DIFF'] = install_pay['AMT_INSTALMENT'] - install_pay['AMT_PAYMENT']

# Days past due and days before due (no negative values)
install_pay['DPD'] = install_pay['DAYS_ENTRY_PAYMENT'] - install_pay['DAYS_INSTALMENT']
install_pay['DBD'] = install_pay['DAYS_INSTALMENT'] - install_pay['DAYS_ENTRY_PAYMENT']
install_pay['DPD'] = install_pay['DPD'].apply(lambda x: x if x > 0 else 0)
install_pay['DBD'] = install_pay['DBD'].apply(lambda x: x if x > 0 else 0)

# # Features: Perform aggregations
# aggregations = {
#     'NUM_INSTALMENT_VERSION': ['nunique'],
#     'DPD': ['max', 'mean', 'sum'],
#     'DBD': ['max', 'mean', 'sum'],
#     'PAYMENT_PERC': ['max', 'mean', 'sum'],
#     'PAYMENT_DIFF': ['max', 'mean', 'sum'],
#     'AMT_INSTALMENT': ['max', 'mean', 'sum'],
#     'AMT_PAYMENT': ['min', 'max', 'mean', 'sum'],
#     'DAYS_ENTRY_PAYMENT': ['max', 'mean', 'sum']
# }

# On enlève les variances
# Features: Perform aggregations
aggregations = {
    'NUM_INSTALMENT_VERSION': ['nunique'],
    'DPD': ['max', 'mean', 'sum'],
    'DBD': ['max', 'mean', 'sum'],
    'PAYMENT_PERC': ['max', 'mean', 'sum', 'var'],
    'PAYMENT_DIFF': ['max', 'mean', 'sum', 'var'],
    'AMT_INSTALMENT': ['max', 'mean', 'sum'],
    'AMT_PAYMENT': ['min', 'max', 'mean', 'sum'],
    'DAYS_ENTRY_PAYMENT': ['max', 'mean', 'sum']
}

for cat in cat_cols:
    aggregations[cat] = ['mean']
ins_agg = install_pay.groupby('SK_ID_CURR').agg(aggregations)
ins_agg.columns = pd.Index(['INSTAL_' + e[0] + "_" + e[1].upper() for e in ins_agg.columns.tolist()])

# Count installments accounts
ins_agg['INSTAL_COUNT'] = install_pay.groupby('SK_ID_CURR').size()

In [50]:
ins_agg.isnull().sum()

INSTAL_NUM_INSTALMENT_VERSION_NUNIQUE      0
INSTAL_DPD_MAX                             0
INSTAL_DPD_MEAN                            0
INSTAL_DPD_SUM                             0
INSTAL_DBD_MAX                             0
INSTAL_DBD_MEAN                            0
INSTAL_DBD_SUM                             0
INSTAL_PAYMENT_PERC_MAX                    0
INSTAL_PAYMENT_PERC_MEAN                   0
INSTAL_PAYMENT_PERC_SUM                    0
INSTAL_PAYMENT_PERC_VAR                  972
INSTAL_PAYMENT_DIFF_MAX                    0
INSTAL_PAYMENT_DIFF_MEAN                   0
INSTAL_PAYMENT_DIFF_SUM                    0
INSTAL_PAYMENT_DIFF_VAR                  972
INSTAL_AMT_INSTALMENT_MAX                  0
INSTAL_AMT_INSTALMENT_MEAN                 0
INSTAL_AMT_INSTALMENT_SUM                  0
INSTAL_AMT_PAYMENT_MIN                     0
INSTAL_AMT_PAYMENT_MAX                     0
INSTAL_AMT_PAYMENT_MEAN                    0
INSTAL_AMT_PAYMENT_SUM                     0
INSTAL_DAY

In [51]:
ins_agg.shape

(339587, 26)

In [52]:
ins_agg.replace([np.inf, -np.inf], np.nan, inplace=True)
ins_agg.dropna(inplace=True)

ins_agg.shape

(338615, 26)

## Credit card balance (6/6)

In [53]:
# ccb_cols_drop = []

# ccb_cols_voir = []

# Pour l'instant on supprime les colonnes où on ne sait pas trop
# ccb = ccb.drop(columns = ins_cols_drop)
# ccb = ccb.drop(columns = ins_cols_voir)

In [54]:
ccb.isnull().sum()

SK_ID_PREV                         0
SK_ID_CURR                         0
MONTHS_BALANCE                     0
AMT_BALANCE                        0
AMT_CREDIT_LIMIT_ACTUAL            0
AMT_DRAWINGS_ATM_CURRENT      749816
AMT_DRAWINGS_CURRENT               0
AMT_DRAWINGS_OTHER_CURRENT    749816
AMT_DRAWINGS_POS_CURRENT      749816
AMT_INST_MIN_REGULARITY       305236
AMT_PAYMENT_CURRENT           767988
AMT_PAYMENT_TOTAL_CURRENT          0
AMT_RECEIVABLE_PRINCIPAL           0
AMT_RECIVABLE                      0
AMT_TOTAL_RECEIVABLE               0
CNT_DRAWINGS_ATM_CURRENT      749816
CNT_DRAWINGS_CURRENT               0
CNT_DRAWINGS_OTHER_CURRENT    749816
CNT_DRAWINGS_POS_CURRENT      749816
CNT_INSTALMENT_MATURE_CUM     305236
NAME_CONTRACT_STATUS               0
SK_DPD                             0
SK_DPD_DEF                         0
dtype: int64

In [55]:
for col in ccb.columns:
    ccb[col] = ccb[col].fillna(0)

In [56]:
# Les 749816, on peut remplacer par 0, ce sont les individus qui n'ont pas utilisé leur crédit.

In [57]:
# Pour 262650 lignes, il s'agit des individus qui n'ont pas utilisé leur crédit
# On s'intéresse au reste.

In [58]:
ccb.dtypes

SK_ID_PREV                      int64
SK_ID_CURR                      int64
MONTHS_BALANCE                  int64
AMT_BALANCE                   float64
AMT_CREDIT_LIMIT_ACTUAL         int64
AMT_DRAWINGS_ATM_CURRENT      float64
AMT_DRAWINGS_CURRENT          float64
AMT_DRAWINGS_OTHER_CURRENT    float64
AMT_DRAWINGS_POS_CURRENT      float64
AMT_INST_MIN_REGULARITY       float64
AMT_PAYMENT_CURRENT           float64
AMT_PAYMENT_TOTAL_CURRENT     float64
AMT_RECEIVABLE_PRINCIPAL      float64
AMT_RECIVABLE                 float64
AMT_TOTAL_RECEIVABLE          float64
CNT_DRAWINGS_ATM_CURRENT      float64
CNT_DRAWINGS_CURRENT            int64
CNT_DRAWINGS_OTHER_CURRENT    float64
CNT_DRAWINGS_POS_CURRENT      float64
CNT_INSTALMENT_MATURE_CUM     float64
NAME_CONTRACT_STATUS           object
SK_DPD                          int64
SK_DPD_DEF                      int64
dtype: object

In [59]:
# Preprocess credit_card_balance.csv
cols_ccb_change = ['SK_ID_PREV', 'SK_ID_CURR']
ccb, cat_cols, not_categorical_columns_ccb = preprocessing(ccb, cols_to_change=cols_ccb_change)

# General aggregations
ccb.drop(['SK_ID_PREV'], axis= 1, inplace = True)
# cc_agg = ccb.groupby('SK_ID_CURR').agg(['min', 'max', 'mean', 'sum', 'var'])
cc_agg = ccb.groupby('SK_ID_CURR').agg(['min', 'max', 'mean', 'sum'])
cc_agg.columns = pd.Index(['CC_' + e[0] + "_" + e[1].upper() for e in cc_agg.columns.tolist()])

# Count credit card lines
cc_agg['CC_COUNT'] = ccb.groupby('SK_ID_CURR').size()

In [47]:
cc_agg.isnull().sum().max()

0

In [48]:
cc_agg.shape

(103558, 109)

In [49]:
cc_agg.replace([np.inf, -np.inf], np.nan, inplace=True)
cc_agg.dropna(inplace=True)

cc_agg.shape

(103558, 109)

In [50]:
# df_nan = cc_agg.isnull().sum()
# df_nan = df_nan.sort_values(ascending=False)
# liste_nan = []

# for x, y in zip(df_nan.index, df_nan):
#     liste_nan.append((x, y))

# liste_nan[:30]

## Join

In [61]:
df = df.join(bureau_agg, how='left', on='SK_ID_CURR')
df = df.join(prev_agg, how='left', on='SK_ID_CURR')
df = df.join(pos_agg, how='left', on='SK_ID_CURR')
df = df.join(ins_agg, how='left', on='SK_ID_CURR')
df = df.join(cc_agg, how='left', on='SK_ID_CURR')

In [62]:
for col in df.columns:
    if 'BURO' in col:
        df[col].fillna(0, inplace=True)

    elif 'ACTIVE' in col:
        df[col].fillna(0, inplace=True)

    elif 'CLOSED' in col:
        df[col].fillna(0, inplace=True)

    elif 'EXT_SOURCE' in col:
        df[col].fillna(df[col].median(), inplace=True)

    elif 'PREV' in col:
        df[col].fillna(df[col].median(), inplace=True)

    elif 'REFUSED' in col:
        df[col].fillna(df[col].median(), inplace=True)

    elif 'APPROVED' in col:
        df[col].fillna(df[col].median(), inplace=True)
        
    elif 'POS' in col:
        df[col].fillna(df[col].median(), inplace=True)
        
    elif 'CC' in col:
        df[col].fillna(0, inplace=True)

    elif 'INSTAL' in col:
        df[col].fillna(df[col].median(), inplace=True)

In [63]:
df.shape

(356251, 558)

## Classification

In [66]:
df.duplicated().sum()

0

In [67]:
# Normalement on est bon sur les inf !
for col in df.columns.drop('TARGET'):
    df[col].replace([np.inf, -np.inf], np.nan, inplace=True)
    df[col].dropna(inplace=True)

df.shape

(356251, 558)

In [29]:
# Globalement les mêmes variables ont des valeurs manquantes
# Un peu plus côté train

In [31]:
# # On remplace nos dernières valeurs manquantes par la médiane
# cols_to_fill = ['PAYMENT_RATE', 'ANNUITY_INCOME_PERC', 'AMT_ANNUITY', 'CNT_FAM_MEMBERS', 'INCOME_PER_PERSON']

# for col in cols_to_fill:
#     train_df[col].fillna(train_df[col].median(), inplace = True)
#     test_df[col].fillna(test_df[col].median(), inplace = True)

In [94]:
# print('moyenne :', round(train_df['DAYS_EMPLOYED_PERC'].mean(), 2))
# print('médiane :', round(train_df['DAYS_EMPLOYED_PERC'].median(), 2))
# print('mode :', train_df['DAYS_EMPLOYED_PERC'].mode())

# # Histrogramme de la distribution de la variable
# plt.figure(figsize = (6, 6))
# plt.hist(x = train_df['DAYS_EMPLOYED_PERC'].values, bins = 50, align = 'left', color = '#4d7cab',
#          edgecolor = 'black', linewidth = 1.1)
# plt.title('Distribution de notre colonne "DAYS_EMPLOYED_PERC"')
# plt.xlabel('DAYS_EMPLOYED_PERC')
# plt.ylabel('Count')
# plt.show()

In [33]:
# cols_to_fill_bis = ['DAYS_EMPLOYED_PERC', 'DAYS_EMPLOYED']

# for col in cols_to_fill_bis:
#     train_df[col].fillna(train_df[col].median(), inplace = True)
#     test_df[col].fillna(test_df[col].median(), inplace = True)

In [ ]:
# df.isnull().sum

In [ ]:
# for col in df.columns.drop('TARGET'):
#     df[col].replace([np.inf, -np.inf], np.nan, inplace=True)
#     df[col].dropna(inplace=True)

# df.shape

In [52]:
# Class imbalance (class weight ou SMOTE)

In [53]:
# t0 = time.time()
# # Divide in training/validation and test data
# print("Starting LogicticRegression. Train shape: {}, test shape: {}".format(train_df.shape, test_df.shape))

# # Cross validation model
# # if stratified:
# #     folds = StratifiedKFold(n_splits= num_folds, shuffle=True, random_state=24)
# # else:
# folds = KFold(n_splits= 10, shuffle=True, random_state=24)
    
# # Create arrays and dataframes to store results
# oof_preds = np.zeros(train_df.shape[0])
# sub_preds = np.zeros(test_df.shape[0])
# feature_importance_df = pd.DataFrame()
# feats = [f for f in train_df.columns if f not in ['TARGET','SK_ID_CURR','SK_ID_BUREAU','SK_ID_PREV','index']]

# X = train_df.drop('TARGET', axis = 1)
# y = train_df['TARGET']

# clf = LogisticRegression(class_weight={0:1, 1:5}, random_state=24)

# clf.fit(X, y)

# y_proba_pred = clf.predict_proba(test_df.drop('TARGET', axis = 1))
# y_pred = clf.predict(test_df.drop('TARGET', axis = 1))
# t1 = time.time()
# time_round = np.round(t1 - t0, 2)
# print(f'Time : {time_round} secondes')

## Corrélations

In [69]:
df.shape

(356251, 558)

In [73]:
train_df = df[df['TARGET'].notnull()]
test_df = df[df['TARGET'].isnull()]

In [85]:
train_df['TARGET'] = train_df['TARGET'].astype('int64')

In [86]:
# Find correlations with the target and sort
correlations = train_df.corr()['TARGET'].sort_values()

In [87]:
# Display correlations
print('Most Positive Correlations:\n', correlations.tail(10))
print('\nMost Negative Correlations:\n', correlations.head(10))

Most Positive Correlations:
 BURO_DAYS_CREDIT_MIN                                0.073969
PREV_NAME_CONTRACT_STATUS_Refused_MEAN              0.077903
DAYS_BIRTH                                          0.078242
BURO_DAYS_CREDIT_MEAN                               0.080145
TARGET                                              1.000000
PREV_NAME_GOODS_CATEGORY_House Construction_MEAN         NaN
CC_NAME_CONTRACT_STATUS_Approved_MIN                     NaN
CC_NAME_CONTRACT_STATUS_Demand_MIN                       NaN
CC_NAME_CONTRACT_STATUS_Refused_MIN                      NaN
CC_NAME_CONTRACT_STATUS_Sent proposal_MIN                NaN
Name: TARGET, dtype: float64

Most Negative Correlations:
 EXT_SOURCE_2                              -0.160295
EXT_SOURCE_3                              -0.156026
EXT_SOURCE_1                              -0.098875
BURO_CREDIT_ACTIVE_Closed_MEAN            -0.076500
PREV_CODE_REJECT_REASON_XAP_MEAN          -0.074356
PREV_NAME_CONTRACT_STATUS_Approved_MEAN   

Normal d'avoir des NaNs, on a quasi pas de variation dans certaines colonnes.

cf. : https://stackoverflow.com/questions/22655667/dataframe-correlation-produces-nan-although-its-values-are-all-integers

Réduction de dimension ?

In [117]:
# from sklearn.decomposition import PCA

# print("Dimensions dataset avant réduction PCA : ", train_df.shape)
# pca = PCA(n_components=0.99)
# feat_pca= pca.fit_transform(train_df)
# print("Dimensions dataset après réduction PCA : ", feat_pca.shape)

Dimensions dataset avant réduction PCA :  (307507, 571)
Dimensions dataset après réduction PCA :  (307507, 1)


In [86]:
# for col in df.columns.drop([list_categorical_columns, 'SK_ID_CURR', 'TARGET', 'CODE_GENDER', 'FLAG_OWN_CAR'
#                            'FLAG_OWN_REALTY']):
#     scaler = StandardScaler()
#     df[col] = scaler.fit_transform(df[col])

In [ ]:
# train_df = df[df['TARGET'].notnull()]
# test_df = df[df['TARGET'].isnull()]

In [88]:
# On regarde l'équilibre de nos classes
train_df['TARGET'].value_counts()

0    282682
1     24825
Name: TARGET, dtype: int64

In [89]:
X = train_df.drop('TARGET', axis = 1)
y = train_df['TARGET']
folds = StratifiedKFold(n_splits=5, shuffle=True, random_state=24)
cv = folds.split(X, y)
cv = list(cv)

In [92]:
t0 = time.time()

oof_preds = np.zeros(train_df.shape[0])
sub_preds = np.zeros(test_df.shape[0])
feature_importance_df = pd.DataFrame()
feats = [f for f in train_df.columns if f not in ['TARGET','SK_ID_CURR','SK_ID_BUREAU','SK_ID_PREV','index']]

# On sépare notre train en train et validation
for n_fold, (train_idx, valid_idx) in enumerate(cv):
    train_x, train_y = train_df[feats].iloc[train_idx], train_df['TARGET'].iloc[train_idx]
    valid_x, valid_y = train_df[feats].iloc[valid_idx], train_df['TARGET'].iloc[valid_idx]
    
    clf = LogisticRegression(class_weight={0:1, 1:10}, max_iter=500, random_state=24)
    clf.fit(train_x, train_y)
    
    oof_preds[valid_idx] = clf.predict_proba(valid_x)[:, 1]
    sub_preds += clf.predict_proba(test_df[feats])[:, 1] / folds.n_splits
    
    
    fold_importance_df = pd.DataFrame()
    fold_importance_df["feature"] = feats
    
    feature_importances = []
    for _, v in enumerate(clf.coef_[0]):
        feature_importances.append(v)
    
    fold_importance_df["importance"] = feature_importances
    fold_importance_df["fold"] = n_fold + 1
    feature_importance_df = pd.concat([feature_importance_df, fold_importance_df], axis=0)
    print('Fold %2d AUC : %.6f' % (n_fold + 1, roc_auc_score(valid_y, oof_preds[valid_idx])))

t1 = time.time()
time_round = np.round(t0 - t1, 2)
print(f'Time : {time_round} secondes')

Fold  1 AUC : 0.602041


C:\Users\victo\AppData\Roaming\Python\Python38\site-packages\sklearn\linear_model\_logistic.py:814: ConvergenceWarning: lbfgs failed to converge (status=1):
STOP: TOTAL NO. of ITERATIONS REACHED LIMIT.

Increase the number of iterations (max_iter) or scale the data as shown in:
    https://scikit-learn.org/stable/modules/preprocessing.html
Please also refer to the documentation for alternative solver options:
    https://scikit-learn.org/stable/modules/linear_model.html#logistic-regression
  n_iter_i = _check_optimize_result(


Fold  2 AUC : 0.602395


C:\Users\victo\AppData\Roaming\Python\Python38\site-packages\sklearn\linear_model\_logistic.py:814: ConvergenceWarning: lbfgs failed to converge (status=2):
ABNORMAL_TERMINATION_IN_LNSRCH.

Increase the number of iterations (max_iter) or scale the data as shown in:
    https://scikit-learn.org/stable/modules/preprocessing.html
Please also refer to the documentation for alternative solver options:
    https://scikit-learn.org/stable/modules/linear_model.html#logistic-regression
  n_iter_i = _check_optimize_result(


Fold  3 AUC : 0.602213


C:\Users\victo\AppData\Roaming\Python\Python38\site-packages\sklearn\linear_model\_logistic.py:814: ConvergenceWarning: lbfgs failed to converge (status=2):
ABNORMAL_TERMINATION_IN_LNSRCH.

Increase the number of iterations (max_iter) or scale the data as shown in:
    https://scikit-learn.org/stable/modules/preprocessing.html
Please also refer to the documentation for alternative solver options:
    https://scikit-learn.org/stable/modules/linear_model.html#logistic-regression
  n_iter_i = _check_optimize_result(


Fold  4 AUC : 0.554487


C:\Users\victo\AppData\Roaming\Python\Python38\site-packages\sklearn\linear_model\_logistic.py:814: ConvergenceWarning: lbfgs failed to converge (status=1):
STOP: TOTAL NO. of ITERATIONS REACHED LIMIT.

Increase the number of iterations (max_iter) or scale the data as shown in:
    https://scikit-learn.org/stable/modules/preprocessing.html
Please also refer to the documentation for alternative solver options:
    https://scikit-learn.org/stable/modules/linear_model.html#logistic-regression
  n_iter_i = _check_optimize_result(


Fold  5 AUC : 0.669810
Time : -365.62 secondes


In [44]:
# On met nos scores dans le bon DF et on calcule accuracy pour voir ce que ça donne
# Voir la confusion matrix

In [ ]:
# Mettre d'autres metrics

In [60]:
y_pred_train = clf.predict(train_df.drop('TARGET', axis = 1))

In [61]:
accuracy_score(y, y_pred_train)

0.9192701304360551

In [ ]:
df_try = train_df